In [175]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime

#toranoana
#fkey作者來源key.txt
#fcomiclist結果key_toranoana.text

#輸出格式：
#toranoana
#!作者
#!總筆數
#==類別_數量_中文敘述
#類別_品名
#網址

#頁顯示數量
pnum = 30
mlink = 'http://www.toranoana.jp/'

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

#key=作者
key2 = urllib.quote(key.decode('utf8').encode('shift_jis'))#當sjis輸出utf8的url
key3 = urllib.unquote(key2.decode('shift_jis').encode('utf8'))#utf8的url翻sjis
#網址用
key4 = urllib.quote(key)

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'


'''
res = requests.post("http://www.toranoana.jp/cgi-bin/R2/d_search.cgi", data = adict)

網址樣本
http://www.toranoana.jp/cgi-bin/R2/d_search.cgi
?item_kind=0401
&stk=1
&obj=0
&nam=
&mak=
&act=%82%cd%82%e9%82%e9%82%f1
&adl=0
&img=0#圖顯示0不顯示，1顯示
&dys=
&dms=01
&dye=
&dme=01
&bl_fg=0
&ps=1#頁數控制，31 61
&itc=
&ikb=
&gnr=
&mch=
&com=

'''

link = "http://www.toranoana.jp/cgi-bin/R2/d_search.cgi?item_kind=0401&stk=1&obj=0&nam=&mak=&act=" + key2 + "&adl=0&img=0&dys=&dms=01&dye=&dme=01&ps=1&bl_fg=0&itc=&ikb=&gnr=&mch=&com="
cookies = {'_tcuid':'201602230349005234',
               '_tcuid_updated_at':'1456170540395',
               '_tcsid':'201602230349001329',
               '_tcsid_updated_at':'1456170540395',
               '_ga':'GA1.2.1089385032.1456170540',
               'afg':'0'
}#'''作品頁面年齡驗證

res = requests.get(link)

res.encoding =  res.apparent_encoding#亂碼處理
r= res.text
r=r[r.find(u'<!-- MAIN AREA -->'):r.find(u'<!-- /MAIN AREA -->')+len(u'<!-- /MAIN AREA -->')]#搜尋結果擷取
#r=r[r.find(u'<!-- MAIN AREA -->'):r.rfind(u'</table>')+len(u'</table>')]#搜尋結果擷取
#</table>
#res.text[res.text.find(u'<!-- MAIN AREA -->'):res.text.find(u'<!-- /MAIN AREA -->')+len(u'<!-- /MAIN AREA -->')]

#print r

only_a_tags = SoupStrainer("table",class_="f_tbl_9cf",cellspacing="1")#縮小檢索範圍
#only_a_tags = SoupStrainer("tr", class_="TBLdtil")
#only_a_tags = SoupStrainer("td",class_="DTW_td_l")#縮小檢索範圍
soup = BeautifulSoup(r,"html.parser",  parse_only=only_a_tags)
#soup1 = BeautifulSoup(r,"lxml")
#soup = BeautifulSoup(r,"html.parser")#超出lxml緩存，改其他存取
#soup = BeautifulSoup(r,"lxml")
#sou = BeautifulSoup(str(soup,),"lxml",  parse_only=only_a_tags)
#print len(r),len(str(soup)),len(str(soup1))
#print soup,len(str(soup))

#Details"，作品頁目標

def next(page = 2):
    #page=int(page)
    ps=page*pnum+1
    link = "http://www.toranoana.jp/cgi-bin/R2/d_search.cgi?item_kind=0401&stk=1&obj=0&nam=&mak=&act=" + key2 +     "&adl=0&img=0&dys=&dms=01&dye=&dme=01&ps="+str(ps)+"&bl_fg=0&itc=&ikb=&gnr=&mch=&com="
    
    res = requests.get(link)
    res.encoding =  res.apparent_encoding
    r= res.text
    r=r[r.find(u'<!-- MAIN AREA -->'):r.find(u'<!-- /MAIN AREA -->')+len(u'<!-- /MAIN AREA -->')]#搜尋結果擷取
    
    only_a_tags = SoupStrainer("tr", class_="TBLdtil")#縮小處理範圍
    #先html.parser解析與縮小範圍，再以字串給lxml
    #soup = BeautifulSoup(str(BeautifulSoup(res.text,"html.parser",  parse_only=only_a_tags)),"lxml")
    soup = BeautifulSoup(r,"lxml",  parse_only=only_a_tags)
    
    return soup

#全轉半
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring
#半轉全
def B2Q(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    ustring=ustring.lower()
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

#資料儲存
def save(sdict , check=0):
    #print listdata
    for temp in listdata:
        if sdict.get(temp):
            #fout.write(temp.encode('utf8') +sdict[temp].encode('utf8')  + '\n')
            fout.write(sdict[temp].encode('utf8')  + '\n')
    #return

#作品頁面
def finddata(clink):
    blink=mlink+clink
    res = requests.get(blink,cookies=cookies)
    res.encoding =  res.apparent_encoding#亂碼處理
    r= res.text
    r=r[r.find(u'<td class=\"Main\">'):r.find(u'<!--フッターここまで-->')+len(u'<!--フッターここまで-->')]+u'\n</td>'#搜尋結果擷取
    
    only_a_tags = SoupStrainer("table", summary="Details")#縮小檢索範圍
    sou=BeautifulSoup(r,"lxml",  parse_only=only_a_tags)
    
    dname=sou.select('.DetailData_L')[2].text
    draw=sou.select('.DetailData_L')[3].text
    dtype=sou.select('.DetailData_R')[0].text
    ddate=sou.select('.DetailData_R')[1].text
    #案例處理
    #print 'fp',blink,sou
    print dname,draw,dtype,ddate
    #return

#資料處理
def findbook(soup , page = 1):
    a =0
    check = 0
    for tb in soup.select('.TBLdtil'):
        clink= tb.select('a')[0].get('href')[1:]
        cname=tb.select('.noi_c2')[0].text
        cbl=tb.select('.noi_c5')[0].text
        cclub=tb.select('.noi_c3')[0].text
        #print clink,cname,cbl
        #print a,#cclub,
        
        
        if (u'株式会社虎の穴' in cclub) or (u'【とらのあな' in cname):
            check=1
        else:
            cdata=finddata(clink)
        continue
        #sys.exit()################
        #網址
        blink = ''
        blink = sou.select('.blueb')[0].get('href')
        blink = mlink + blink[3:]
        #print blink
        
        #類型
        if u'・' in ctype:
            stype = ctype.split(u'・')
            dtype = stype[0] + ']'
        else:
            dtype = ctype
        
        #品名
        book = cbook
        
        #日期處理
        #print cdata
        cdata = cdata.rstrip()
        if len(cdata) < 8:#無日期
            cdata = '0000/00/00'#填入日期
            check = 4#新作
        data = cdata
        while listdata.count(data):#重複日期判斷
            data = data[:8] + str(int(data[8:]) + 1).rjust(2,'0')#日期+1_十位數填0
        listdata.append(data)
            
        '''要排除的
        cname含虎穴特典
        類型不合
        作者錯誤
        BL
        '''
        
        #寫入dict
        book = dtype + '_' + book + '_' + '\n!' + blink#類形+書名+網址
        if check == 4:
            dict4.setdefault(data,book)
        elif dtype in [u'[BOOKS]',u'[雑誌]',u'[同人]'] :
            if (u'ノベルズ' in book):
                book= book[:book.find(u']')]+u'_ノベルズ]'+book[book.find(u']'):]
            elif (u'文庫' in book):
                book= book[:book.find(u']')]+u'_文庫]'+book[book.find(u']'):]
            dict1.setdefault(data,book)
        elif dtype in[u'[アニメ]']:
            dict2.setdefault(data,book)
        elif dtype in [u'[PCゲーム]',u'[DVDPG]']:
            dict3.setdefault(data,book)
        else:
            dict5.setdefault(data,book)
        
        a = a + 1
        #print '\r',a,
    print '.'
    #print '========'
    #return

########

key=key.lower()
#print soup
pn=0
if soup.select('span'):
    pn=int(soup.select('span')[2].text[:soup.select('span')[2].text.find(u'件')])#資料筆數
    #print pn,type(pn)

'''
成年向け同人誌 -
全年齢対象同人誌 -
同人CG
- 同人ソフト -軟體
- 同人音楽作品 -CD
- 同人グッズ -周邊
#'''

#pn = soup.select('.s_condition')[0].select('b')[0].text
#資料筆數_是否數字
if pn>0:
    print 'ok'
    #print soup.find_all('b')
    
    if pn > pnum:
        print 'BIG'
    
    fout = open('output/'+key.decode('utf8') + '_toranoanav1.txt', 'w')#寫入模式開檔
    fout.write('toranoana\n')#toranoana
    print key , pn , 'num\n========v1'
    time.sleep(1)
    fout.write('!' + key + '\n!總筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d,%H:%M")+'->')
    
    p = 0#頁
    #建空輸出用字典與陣列
    dict1={}#BOOKS,雑誌,同人
    dict2={}#アニメ
    dict3={}#PCゲーム,DVD-PG
    dict4={}#新
    dict5={}#其他
    listdata = []
    
    #資料處理
    while (int(pn) - p * pnum) > 0:
        
        print 'page:' + str(p)
        soup = next(p)#頁
        #print soup.select('.TBLdtil')[2]
        findbook(soup)#資料處理
        time.sleep(1)
        p = p + 1
        sys.exit()################
    #print 'ook\n',listdata
    
    #日期排序
    listdata.sort()
    
    fout.write(strftime("%H:%M")+'\n')
    temp = ''
    #dict1_BOOKS,雑誌,同人輸出
    fout.write('==book_' + str(len(dict1)) +'_BOOKS,雑誌,同人\n')
    save(dict1)
    #dict2_アニメ輸出
    fout.write('==anime_' + str(len(dict2)) +'_アニメ\n')
    save(dict2)
    #dict3_PCゲーム,DVD-PG輸出
    fout.write('==game_' + str(len(dict3)) +'_PCゲーム,DVD-PG\n')
    save(dict3)
    #dict4_新輸出
    fout.write('==new_' + str(len(dict4)) +'_新\n')
    save(dict4)
    #dict5_其他輸出
    fout.write('==other_' + str(len(dict5)) +'_其他\n')
    save(dict5)
    
    #sys.exit()################
    fout.close()
    print 'ok'
elif pn:
    print 'No Date'

#結束讀秒
x=3
while x!=0:
    print x,'..',
    x=x-1
    time.sleep(1)
raw_input("\nPress Any Key To Exit")

ok
BIG
しおこんぶ 36 num
========v1
page:0

しおこんぶ

THE IDOLM@STER
同人誌 2015/12/31

しおこんぶ

THE IDOLM@STER
同人誌 2015/08/16

しおこんぶ

THE IDOLM@STER
同人誌 2015/05/02

しおこんぶ

THE IDOLM@STER
同人誌 2014/12/30

しおコンブ

戦国BASARA
同人誌 2014/09/21

しおこんぶ

THE IDOLM@STER
同人誌 2014/08/17

しおこんぶ

THE IDOLM@STER
同人誌 2013/12/31

しおこんぶ

THE IDOLM@STER
同人誌 2013/08/11

しおこんぶ

ローゼンメイデン
同人誌 2013/04/28

しおこんぶ

俺の妹がこんなに可愛いわけがない
同人誌 2012/12/31

しおこんぶ

ベン・トー
同人誌 2012/08/12

しおこんぶ

アクセルワールド
同人誌 2012/08/12

しおこんぶ

ペルソナ
同人誌 2012/04/30

塩/こんぶ

青の祓魔師
同人誌 2012/01/08

しおこんぶ

ベン・トー
同人誌 2011/12/31

しおこんぶ

VOCALOID
同人誌 2011/10/23
.


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [173]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime
mlink = 'http://www.toranoana.jp/'
c='''
<table cellpadding="0" cellspacing="0" summary="Details" width="100%">
<tr>
<td valign="middle">
<table cellpadding="0" cellspacing="1" width="100%">
<tr>
<td class="DetailSummary_L">タイトル</td>
<td class="DetailData_L" colspan="3">恋の風向き</td>
<td class="DetailData_SubmitArea" rowspan="8">
<img height="0" src="http://wwwcx.toranoana.jp/cf/?C=040030322168&amp;T=648&amp;Z=1&amp;S=0&amp;L=4&amp;O=0&amp;R=%2fmailorder%2farticle%2f04%2f0030%2f32%2f21%2f040030322168%2ehtml&amp;N=%ce%f8%a4%ce%c9%f7%b8%fe%a4%ad&amp;M=LAMINARIA" width="0"/>
■　注文番号　■<br/>
<b>040030322168</b><br/>
価格：<span class="bold">600</span><b>円</b>（＋税）<br/>
<img alt="在庫状況アイコン" border="0" src="/mailorder/common/images/mat/stock4.gif"/>在庫あり<br/>
<form action="/cgi-bin/R5/details.cgi" method="post">
<input name="TAXRATE" type="hidden" value="8"/>
<input name="GNAME" type="hidden" value="恋の風向き"/>
<input name="CODE" type="hidden" value="040030322168"/>
<input name="TANKA" type="hidden" value="600"/>
<input name="REM1" type="hidden" value="0"/>
<input name="REM2" type="hidden" value="1"/>
<input name="REM3" type="hidden" value="0"/>
<input name="ST" type="hidden" value="M"/>
<input name="sk" type="hidden" value=""/>
<select name="KAZU"><option selected="" value="1">1
</option><option value="2">2
</option><option value="3">3
</option><option value="4">4
</option><option value="5">5
</option></select>部
<input type="submit" value="購　入"/>
</form>
<a alt="ほしい!" href="javascript:" onclick="window.open('/cgi-bin/wish_append.cgi?item_code=040030322168&amp;bl_fg=0&amp;sk=','wish_append','width=280,height=320,location=no,toolbar=no');"><img border="0" src="/es/img/hoshii.gif"/></a>
<!--Twitter,Facebook-->
<table style="width:100%; table-layout:fixed; border-collapse:collapse;">
<tr>
<td align="left" style="padding:20px 0px 0px 25px;">
<a class="twitter-share-button" data-count="horizontal" data-dnt="" data-hashtags="同人誌" data-lang="ja" data-related="" data-size="medium" data-text="恋の風向き" data-url="http://www.toranoana.jp/mailorder/article/04/0030/32/21/040030322168.html" data-via="tora_ec_dansei" href="https://twitter.com/share">ツイート</a>
</td>
</tr>
</table>
</td>
</tr>
<tr>
<td class="DetailSummary_L">サークル</td>
<td class="DetailData_L"><a href="/mailorder/cot/circle/54/05/5730323030353534/4c414d494e41524941_01.html">LAMINARIA</a></td>
<td class="DetailSummary_R">種別</td>
<td class="DetailData_R">同人誌</td>
</tr>
<tr>
<td class="DetailSummary_L">主な作家</td>
<td class="DetailData_L">
<a href="/mailorder/cot/author/32/a4b7a4aaa4b3a4f3a4d6_01.html">しおこんぶ</a>
</td>
<td class="DetailSummary_R">発行日</td>
<td class="DetailData_R">2015/08/16</td>
</tr>
<tr>
<td class="DetailSummary_L">カテゴリ</td>
<td class="DetailData_L">
<a href="/mailorder/cot/genre/4/730_01.html">THE IDOLM@STER</a>
</td>
<td class="DetailSummary_R">サイズ</td>
<td class="DetailData_R">B5 32p</td>
</tr>
<td class="DetailSummary_L">メインキャラ</td>
<td class="DetailData_L">高垣楓</td>
<td class="DetailSummary_R">指定</td>
<td class="DetailData_R"><span class="Adult">※18禁</span></td>
<tr>
<td class="DetailSummary_L">コメント</td>
<td class="DetailData_Comment" colspan="3">楓さんがアイドルとしてデビューする以前を描きました、純愛で切ない感じにしてみました。</td>
</tr>
<tr>
<td class="DetailSummary_L">注意事項</td>
<td class="DetailData_Comment" colspan="3">
					      	返品については<a href="/mailorder/guide/guide/guide_061.html" target="_blank">こちら</a>をご覧下さい。<br/>
					      	お届けまでにかかる日数については<a href="/mailorder/guide/guide/guide_041.html" target="_blank">こちら</a>をご覧下さい。<br/>
					      	ご予約注文「えらべる予約」については<a href="/mailorder/guide/guide/guide_037.html" target="_blank">こちら</a>をご覧下さい。<br/>
					      	再販投票については<a href="/mailorder/guide/guide/guide_084.html" target="_blank">こちら</a>をご覧下さい。
					      </td>
</tr>
<tr>
<td class="DetailSummary_L">消費税率に関してのお知らせ</td>
<td class="DetailData_Comment" colspan="3">
									平成26年4月1日からの消費税率変更に伴って、<br/>
									現在の本体価格+5%から本体価格＋新税率に変更となります。<br/>
									消費税率変更後は平成26年4月1日以降の発売商品だけではなく、現在販売中の商品に関しましても新税率の適用となりますことご了承下さい。
					      </td>
</tr>
</table>
</td>
</tr>
</table>
'''

link='http://www.toranoana.jp/mailorder/article/04/0030/32/21/040030322168.html'
payload = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
    'Cookie':'_tcuid=201602230335498253;_tcuid_updated_at=1456169749024;_tcsid=201602230335490499; _tcsid_updated_at=1456169818666; afg=0; _ga=GA1.2.1555831105.1456169749',
    'Referer:http':'//www.toranoana.jp/mailorder/article/04/0030/32/21/040030322168.html'
}
cookies = {'_tcuid':'201602230349005234',
               '_tcuid_updated_at':'1456170540395',
               '_tcsid':'201602230349001329',
               '_tcsid_updated_at':'1456170540395',
               '_ga':'GA1.2.1089385032.1456170540',
               'afg':'0'
}#'''
#cookies = dict(cookies_are='working')
#Cookie:_tcuid=201602230335498253; _tcuid_updated_at=1456169749024; _tcsid=201602230335490499; _tcsid_updated_at=1456169818666; afg=0; _ga=GA1.2.1555831105.1456169749
#res=requests.session()
#re = res.get(link,data=payload)
#res = requests.get(link,cookies=cookies)
res.encoding =  res.apparent_encoding#亂碼處理
#r= res.text

#r=r[r.find(u'<td class=\"Main\">'):r.find(u'<!--フッターここまで-->')+len(u'<!--フッターここまで-->')]+u'\n</td>'#搜尋結果擷取
#print r
#r=u'<table>'+r[r.find(u'<td class=\"Main\">'):r.find(u'<!--フッターここまで-->')+len(u'<!--フッターここまで-->')]+u'\n</td></table>'#搜尋結果擷取
#print type(r)
#print r.find(u'<td class=\"Main\">'),r.find(u'<!--フッターここまで-->')

only_a_tags = SoupStrainer("table", summary="Details")#縮小檢索範圍
#class="Main"
#soup=BeautifulSoup(r,"html.parser")#,  parse_only=only_a_tags)
#soup=BeautifulSoup(r,"lxml",  parse_only=only_a_tags)
soup=BeautifulSoup(c,"lxml")#,  parse_only=only_a_tags)

#print len(res.text),len(str(soup)),len(str(sou))
#print soup
print len(soup.select('.DetailData_L')),soup.select('.DetailData_L')[0]
print len(soup.select('.DetailData_R')),soup.select('.DetailData_R')[0]

print 'name',soup.select('.DetailData_L')[2].text
print 'o',soup.select('.DetailData_L')[3].text
print 'type',soup.select('.DetailData_R')[0].text
print 'date',soup.select('.DetailData_R')[1].text
#print soup.select('table')[3]
#print re.text

5 <td class="DetailData_L" colspan="3">恋の風向き</td>
4 <td class="DetailData_R">同人誌</td>
name 
しおこんぶ

o 
THE IDOLM@STER

type 同人誌
date 2015/08/16
